In [1]:
from pathlib import Path
import psycopg2
import time
from tqdm import tqdm

# ----------------------------------
# Paths
# ----------------------------------
WEATHER_LANDING_DIR = Path("/app/data/landing/weather")
csv_files = sorted(WEATHER_LANDING_DIR.glob("*.csv"))

total_files = len(csv_files)
print(f"📦 Files to ingest: {total_files}")

# ----------------------------------
# DB connection
# ----------------------------------
conn = psycopg2.connect(
    dbname="etl_db",
    user="postgres",
    password="postgres",
    host="postgres",
    port=5432,
)
cur = conn.cursor()

# ----------------------------------
# Ingest with timing
# ----------------------------------
start_time = time.perf_counter()
completed = 0

for csv_path in tqdm(csv_files, desc="Ingesting weather files"):
    with open(csv_path, "r") as f:
        cur.copy_expert(
            """
            COPY bronze.weather_daily (
                station_id,
                obs_date,
                element,
                value,
                m_flag,
                q_flag,
                s_flag
            )
            FROM STDIN
            WITH CSV HEADER
            """,
            f
        )

    completed += 1

conn.commit()

elapsed = time.perf_counter() - start_time

# ----------------------------------
# Summary
# ----------------------------------
print("✅ Weather ingestion complete")
print(f"📦 Files ingested: {completed}/{total_files}")
print(f"⏱️ Total time: {elapsed:.2f} seconds")
print(f"⚡ Files/sec: {completed / elapsed:.2f}")

cur.close()
conn.close()


📦 Files to ingest: 3651


Ingesting weather files: 100%|██████████| 3651/3651 [00:59<00:00, 61.63it/s] 

✅ Weather ingestion complete
📦 Files ingested: 3651/3651
⏱️ Total time: 59.25 seconds
⚡ Files/sec: 61.62


In [7]:
# from sqlalchemy import text

# with engine.begin() as conn:
#     conn.execute(text("TRUNCATE TABLE bronze.weather_daily"))
